<h2>Multiple fitting with Differential Evolution (DE) and uncertainty estimation via Metropolis-Hasting</h2>

In [20]:
import numpy as np
import random
from collections import namedtuple
from typing import NamedTuple

In [21]:
R=1.0e7
S=1.5
P=0.5
D=0.0

boundR = (1e6, 1e8)
boundS = (0.5, 2.0)
boundP = (0.3, 1.5)
boundD = (0.0, 0.1)

bounds = (boundR, boundS, boundP, boundD)
guess = (R, S, P, D)

We have the following experimental data that has been categorized by each correspondent series

In [22]:
principal = ((3, 5.86175E-7, 1e-8),
             (4, 3.3003E-7, 1e-8))
sharp = ((5, 6.1435E-7, 1e-8),
         (5, 6.1381E-7, 1e-8),
         (6, 5.1326E-7, 1e-8),   
         (6, 5.1284E-7, 1e-8),   
         (7, 4.7319E-7, 1e-8),   
         (7, 4.7294E-7, 1e-8),   
         (8, 4.5707E-7, 1e-8))
diffuse = ((4, 5.6722E-7, 1e-8),
          (4, 5.6661E-7, 1e-8),
          (5, 4.9618E-7, 1e-8),
          (5, 4.9576E-7, 1e-8),    
          (6, 4.6486E-7, 1e-8),
          (6, 4.6450E-7, 1e-8),
          (7, 4.4856E-7, 1e-8))
balmer = ((3, 6.5930E-7, 1e-9),
          (4, 4.8580E-7, 1e-9),
          (5, 4.3415e-7, 1e-9),
          (6, 4.1056e-7, 1e-9),
          (7, 3.9760e-7, 1e-9))

In [23]:
principal = np.array(principal)
sharp = np.array(sharp)
diffuse = np.array(diffuse)
balmer = np.array(balmer)

In [24]:
Np = len(principal)
Ns = len(sharp) 
Nd = len(diffuse)
Nb = len(balmer)

In [25]:
series = namedtuple('series', ('name', 
                               'level', 
                               'wavelength', 
                               'uncertainty'))
principal_data = series('principal', 
                       principal[:, 0], 
                       principal[:, 1], 
                       principal[:, 2])
sharp_data = series('sharp', 
                   sharp[:, 0], 
                   sharp[:, 1], 
                   sharp[:, 2])
diffuse_data = series('diffuse', 
                     diffuse[:, 0], 
                     diffuse[:, 1], 
                     diffuse[:, 2])
balmer_data = series('balmer', 
                    balmer[:, 0], 
                    balmer[:, 1], 
                    balmer[:, 2])

In [26]:
pop = np.random.rand(4, 10)
pop = np.array([bounds[ii][0] + pop[ii] * (bounds[ii][1] - bounds[ii][0]) for ii in range(len(pop))])
if guess:
    pop[:, 0] = np.array(guess)
pop = pop.T

In [27]:
pop

array([[1.00000000e+07, 1.50000000e+00, 5.00000000e-01, 0.00000000e+00],
       [8.37398242e+07, 1.73470457e+00, 1.15299980e+00, 9.69287346e-03],
       [3.94025382e+07, 1.80348884e+00, 4.44824829e-01, 9.32687924e-03],
       [9.78601901e+07, 9.57366402e-01, 1.24680825e+00, 9.40534729e-03],
       [8.15212964e+07, 1.58403409e+00, 4.52265683e-01, 1.13696470e-03],
       [4.06444038e+06, 1.23490441e+00, 1.25532034e+00, 4.91614891e-02],
       [8.94869645e+07, 5.09537953e-01, 3.32477546e-01, 6.86524762e-02],
       [6.56986386e+07, 1.38854405e+00, 7.85147025e-01, 5.54896288e-02],
       [1.18785231e+07, 1.99859913e+00, 9.80983915e-01, 4.11154655e-02],
       [7.03176635e+07, 1.36915093e+00, 1.47923592e+00, 6.10570903e-02]])

In [28]:
def func_obj(r, cor1, cor2, n, yraw, stdraw):
    y = 1/ (r * ((3 - cor1) ** (-2) - (n - cor2) ** (-2))) 
    err = np.square(yraw - y) / np.square(stdraw) #chi-square goodness of fit ((MSWD))
    return err

In [29]:
data_collection = (principal_data, sharp_data, diffuse_data, balmer_data)
sizes = (Np, Ns, Nd, Nb)

In [30]:
def least_error_idx(pop: np.ndarray, data_collection: list[NamedTuple], sizes : tuple[int]) -> tuple[int, np.ndarray]:
    '''
    Identify the element from the population with the least error.
    Returns the index of the element with the least error.
    '''
    error = np.zeros(len(pop))
    for idx, ind in enumerate(pop):
        error[idx] = compute_error(ind, data_collection, sizes)
    return np.argmin(error), min(error) 

In [32]:
def compute_error(ind: np.ndarray, data_collection: list[NamedTuple], sizes: tuple[int]) -> int:
    '''
    Computes the error function, which is the chi-square goodness of fit
    This is the object function that we are trying to minimize.
    '''
    error = 0
    for data in data_collection:
        if data.name ==  'principal':
            for record in range(sizes[0]):
                error += func_obj(ind[0],
                                  ind[1],
                                  ind[2],
                                  data.level[record],
                                  data.wavelength[record],
                                  data.uncertainty[record])
        elif data.name ==  'sharp':
            for record in range(sizes[1]):
                error += func_obj(ind[0],
                                  ind[2], 
                                  ind[1], 
                                  data.level[record], 
                                  data.wavelength[record], 
                                  data.uncertainty[record])
        elif data.name ==  'diffuse':
            for record in range(sizes[2]):
                error += func_obj(ind[0], 
                                  ind[2], 
                                  ind[3], 
                                  data.level[record], 
                                  data.wavelength[record], 
                                  data.uncertainty[record])
        elif data.name ==  'balmer':
            for record in range(sizes[3]):
                error += func_obj(ind[0],
                                  1, 
                                  0, 
                                  data.level[record], 
                                  data.wavelength[record], 
                                  data.uncertainty[record])
    return np.sqrt(error)

In [33]:
def diff_evolution(pop: np.ndarray[np.ndarray], niter: int, kmut: int, kcross: int, data_collection: list[NamedTuple],
                   sizes: tuple[int]) -> tuple[np.ndarray, int]:
    idx_list = tuple(range(len(pop)))
    lsterr_idx, lst_err = least_error_idx(pop, data_collection, sizes)
    bestfit = pop[lsterr_idx]
    for iter in tqdm(range(niter)):  
        for i, ind in enumerate(pop):
            rng_idx = np.random.choice(idx_list, 2, replace = False)
            trial = bestfit + kmut * (pop[rng_idx[0]] - pop[rng_idx[1]])
            cross = np.concatenate((np.random.rand(3) <= kcross, np.array([True])))
            trial = np.where(cross, trial, ind)
            for j, param in enumerate(trial):
                if not (param >= bounds[j][0] and param <= bounds[j][1]):
                     trial[j] = pop[:, j].min() + np.random.uniform() * (pop[:, j].max() - pop[:, j].min())
            trial_err = compute_error(trial, data_collection, sizes)
            if trial_err < compute_error(ind, data_collection, sizes):
                pop[i] = trial
                if trial_err < lst_err:
                    bestfit = trial
                    lst_err = trial_err
                    yield bestfit, lst_err

In [34]:
from tqdm import tqdm

niter = 50000
kmut = 0.2
kcross = 0.6

res = diff_evolution(pop, niter, kmut, kcross, data_collection, sizes)

In [35]:
best, err = zip(*res)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:55<00:00, 433.56it/s]


In [36]:
best

(array([1.07844934e+07, 1.50000000e+00, 1.02629247e+00, 4.38320986e-02]),
 array([1.08626677e+07, 1.50508487e+00, 1.01215506e+00, 4.39497073e-02]),
 array([1.09765725e+07, 1.48821924e+00, 1.01143180e+00, 4.39954186e-02]),
 array([1.09298220e+07, 1.51364358e+00, 9.80983915e-01, 4.41750774e-02]),
 array([1.09769107e+07, 1.50404621e+00, 9.75588071e-01, 4.41315758e-02]),
 array([1.09151315e+07, 1.50289230e+00, 9.69125569e-01, 4.41902310e-02]),
 array([1.09121935e+07, 1.50074205e+00, 9.66753899e-01, 4.41932617e-02]),
 array([1.09103304e+07, 1.50377053e+00, 9.56855500e-01, 4.41702633e-02]),
 array([1.09295513e+07, 1.50289230e+00, 9.54220884e-01, 4.41666247e-02]),
 array([1.09177831e+07, 1.50309157e+00, 9.53402717e-01, 4.41777496e-02]),
 array([1.09310992e+07, 1.48659596e+00, 9.57149231e-01, 4.41803470e-02]),
 array([1.09193309e+07, 1.48679523e+00, 9.56331065e-01, 4.41914718e-02]),
 array([1.09190328e+07, 1.48225289e+00, 9.50827992e-01, 4.41911724e-02]),
 array([1.09193309e+07, 1.47729610e+00

TODO: implement metropolis hasting

In [10]:
with open('exemplo.par', 'r') as param_file:
    filetxt = param_file.readlines()

In [16]:
filetxt

['Np= 2             numero de linhas de Principal, Nitida, Difusa, Balmer (max 15)\n',
 'Nn= 7\n',
 'Nd= 7\n',
 'Nb= 0\n',
 'R= 1.0e7          valores iniciais de R, S, P, D\n',
 'S= 1.5\n',
 'P= 0.5\n',
 'D= 0.0\n',
 'minR= 1e6         limites inferior e superior de R, S, P, D\n',
 'maxR= 1e8\n',
 'minS= 0.5\n',
 'maxS= 2.0\n',
 'minP= 0.3\n',
 'maxP= 1.5\n',
 'minD= 0.0\n',
 'maxD= 0.1\n',
 'Niteracao= 500000   numero máximo de iteracoes\n',
 'Nfilhos= 20        numero de conjuntos de parametros em cada geracao  (max 40)\n',
 'Kmutacao= 0.2      parametro de mutacao  (entre 0 e 1)\n',
 'Kconverge= 0.6     fator de convergencia para as geracoes sucessivas (entre 0 e 1)\n',
 'crit_erro= 1e-5    criterio de parada, variacao minima do erro relativo (ainda não está em uso) \n',
 'crit_iter= 10000    criterio de parada, numero maximo de iteracoes sem evolucao\n',
 'Principal=  \n',
 '3 5.86175E-7  1e-8           Np linhas da serie principal e incertezas (em m)\n',
 '4 3.3003E-7  1e-8   \n'